# Kaggle Titanic: Applying Neural Net to predict Fatalities and Survivors

In this program we will process the data, create Neural Network and obtain accuracies as well compare it with Decision Tree accuracy.

## Importing required Libraries and Loading data

In [374]:
import os
import subprocess
import numpy as np
import pandas as pd
import math
import matplotlib.patches as patches
import matplotlib.pylab as plt
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree
from sklearn import metrics
from sklearn import datasets
from IPython.display import Image
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from astropy.table import Table
# from sklearn.cross_validation import  cross_val_score

%matplotlib inline
data = pd.read_csv('Titanic.csv',header='infer',parse_dates=True, infer_datetime_format=True);
data.head(3)
data.dropna(axis=0, how='any')
data_filtered = data[~np.isnan(data['age'])]
data_filtered.describe()
data.describe()

,Unnamed: 0,survived,age,sibsp,parch,fare,body
count,1309.000000,1309.000000,1046.000000,1309.000000,1309.000000,1308.000000,121.000000
mean,655.000000,0.381971,29.881135,0.498854,0.385027,33.295479,160.809917
std,378.020061,0.486055,14.413500,1.041658,0.865560,51.758669,97.696922
min,1.000000,0.000000,0.166700,0.000000,0.000000,0.000000,1.000000
25%,328.000000,0.000000,21.000000,0.000000,0.000000,7.895800,72.000000
50%,655.000000,0.000000,28.000000,0.000000,0.000000,14.454200,155.000000
75%,982.000000,1.000000,39.000000,1.000000,0.000000,31.275000,256.000000
max,1309.000000,1.000000,80.000000,8.000000,9.000000,512.329224,328.000000


## Converting Gender to boolean value 0 and 1 to make processing easy

In [2]:
def sex_to_numeric(x):
    if x=='female':
        return 0
    if x=='male':
        return 1
data_filtered['sex'] = data_filtered['sex'].apply(sex_to_numeric)

/Users/khushalidave/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## This function converts string to integer for example: 1st to 1 which will make processing easy for attribute pclass which has values 1st , 2nd

In [3]:
def encode_target(df, target_column):
    """Add column to df with integers for the target.

    Args
    ----
    df -- pandas DataFrame.
    target_column -- column to map to int, producing
                     new Target column.

    Returns
    -------
    df_mod -- modified DataFrame.
    targets -- list of target names.
    """
    df_mod = df.copy()
    targets = df_mod[target_column].unique()
    map_to_int = {name: n for n, name in enumerate(targets)}
    df_mod["Target"] = df_mod[target_column].replace(map_to_int)

    return (df_mod, targets)


In [4]:
df2, targets = encode_target(data_filtered, "pclass")
df2 = df2.drop(["pclass"],axis=1)
df2["pclass"] = df2["Target"]
df2.tail(10)

,Unnamed: 0,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,Target,pclass
1295,1296,0,"Windelov, Mr. Einar",1,21.0,0,0,SOTON/OQ 3101317,7.2500,NaN,Southampton,NaN,NaN,NaN,2,2
1296,1297,0,"Wirz, Mr. Albert",1,27.0,0,0,315154,8.6625,NaN,Southampton,NaN,131.0,NaN,2,2
1298,1299,0,"Wittevrongel, Mr. Camille",1,36.0,0,0,345771,9.5000,NaN,Southampton,NaN,NaN,NaN,2,2
1299,1300,0,"Yasbeck, Mr. Antoni",1,27.0,1,0,2659,14.4542,NaN,Cherbourg,C,NaN,NaN,2,2
1300,1301,1,"Yasbeck, Mrs. Antoni (Selini Al",0,15.0,1,0,2659,14.4542,NaN,Cherbourg,NaN,NaN,NaN,2,2
1301,1302,0,"Youseff, Mr. Gerious",1,45.5,0,0,2628,7.2250,NaN,Cherbourg,NaN,312.0,NaN,2,2
1304,1305,0,"Zabour, Miss. Hileni",0,14.5,1,0,2665,14.4542,NaN,Cherbourg,NaN,328.0,NaN,2,2
1306,1307,0,"Zakarian, Mr. Mapriededer",1,26.5,0,0,2656,7.2250,NaN,Cherbourg,NaN,304.0,NaN,2,2
1307,1308,0,"Zakarian, Mr. Ortin",1,27.0,0,0,2670,7.2250,NaN,Cherbourg,NaN,NaN,NaN,2,2
1308,1309,0,"Zimmerman, Mr. Leo",1,29.0,0,0,315082,7.8750,NaN,Southampton,NaN,NaN,NaN,2,2


In [5]:
data_filtered.head(10)


,Unnamed: 0,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1st,1,"Allen, Miss. Elisabeth Walton",0,29.0000,0,0,24160,211.337494,B5,Southampton,2,NaN,"St Louis, MO"
1,2,1st,1,"Allison, Master. Hudson Trevor",1,0.9167,1,2,113781,151.550003,C22 C26,Southampton,11,NaN,"Montreal, PQ / Chesterville, ON"
2,3,1st,0,"Allison, Miss. Helen Loraine",0,2.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,4,1st,0,"Allison, Mr. Hudson Joshua Crei",1,30.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,5,1st,0,"Allison, Mrs. Hudson J C (Bessi",0,25.0000,1,2,113781,151.550003,C22 C26,Southampton,NaN,NaN,"Montreal, PQ / Chesterville, ON"
5,6,1st,1,"Anderson, Mr. Harry",1,48.0000,0,0,19952,26.549999,E12,Southampton,3,NaN,"New York, NY"
6,7,1st,1,"Andrews, Miss. Kornelia Theodos",0,63.0000,1,0,13502,77.958298,D7,Southampton,10,NaN,"Hudson, NY"
7,8,1st,0,"Andrews, Mr. Thomas Jr",1,39.0000,0,0,112050,0.000000,A36,Southampton,NaN,NaN,"Belfast, NI"
8,9,1st,1,"Appleton, Mrs. Edward Dale (Cha",0,53.0000,2,0,11769,51.479198,C101,Southampton,D,NaN,"Bayside, Queens, NY"
9,10,1st,0,"Artagaveytia, Mr. Ramon",1,71.0000,0,0,PC 17609,49.504200,NaN,Cherbourg,NaN,22.0,"Montevideo, Uruguay"


## Getting 4 columns which we think are useful for prediction

In [6]:
X = df2[['pclass','sex','age','sibsp']]
Y = df2['survived']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=.60)

In [7]:
X_train.head(10)

,pclass,sex,age,sibsp
1108,2,1,19.0000,0
569,1,1,14.0000,0
563,1,1,35.0000,0
1240,2,1,0.4167,0
760,2,1,30.0000,0
963,2,1,36.0000,0
696,2,0,21.0000,0
361,1,0,22.0000,1
260,0,0,40.0000,0
648,2,1,21.0000,0


## Creating Neural Network with best attributes

In [359]:
NN_arch = [(100,100),(10,10), (20,20), (30,30),(40,40),(50,50)]
regularization = [0.000001, 0.000003, 0.00001, 0.00003, 0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1]
accuracy = np.zeros((len(NN_arch),len(regularization)))
#for i in range(len(NN_arch)):
#    for j in range(len(regularization)):
#        clf = MLPClassifier(hidden_layer_sizes=NN_arch[i],solver='adam', activation='logistic', alpha=regularization[j],max_iter=2000)

#        clf.fit(X_train, np.ravel(Y_train))

#        accuracy[i,j] = clf.score(X_train,Y_train)
#print(accuracy)

In [360]:
# Choose the most accurate combination of NN_arch/regularization
id = np.unravel_index(accuracy.argmax(),accuracy.shape)
# Define the classifier
clf = MLPClassifier(hidden_layer_sizes=NN_arch[id[1]],solver='adam', activation='logistic', alpha=regularization[id[1]],max_iter=20000)
# Fit now the whole training set
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

clf.fit(X_train, np.ravel(Y_train))

MLPClassifier(activation='logistic', alpha=1e-06, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 100), learning_rate='constant',
       learning_rate_init=0.001, max_iter=20000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [361]:
in_sample = pd.DataFrame(list(zip(clf.predict(X_test), Y_train)), columns=['predicted', 'actual'])


## Calculating Accuracies for Fatalities and Survivors

In [362]:
in_sample_survivor = in_sample[in_sample.actual == 1]
total_survivor = in_sample_survivor.count().actual
print("total_survivor == %.3f" %total_survivor)

in_sample_survivor_success = in_sample_survivor[in_sample_survivor.predicted == 1]
total_survivor_success = in_sample_survivor_success.count().actual
print("total_survivor_success == %.3f" %total_survivor_success)

in_sample_survivor_accuracy = total_survivor_success/ total_survivor
print("in_sample_survivor_accuracy == %.3f" %in_sample_survivor_accuracy )


total_survivor == 165.000
total_survivor_success == 75.000
in_sample_survivor_accuracy == 0.455


In [363]:
in_sample_fatalities = in_sample[in_sample.actual == 0]
total_fatalities = in_sample_fatalities.count().actual
print("total_fatalities == %.3f" %total_fatalities)

in_sample_fatalities_success = in_sample_fatalities[in_sample_fatalities.predicted == 0]
total_fatalities_success = in_sample_fatalities_success.count().actual
print("total_fatalities_success == %.3f" %total_fatalities_success)

in_sample_fatalities_accuracy = total_fatalities_success/ total_fatalities
print("in_sample_fatalities_accuracy == %.3f" %in_sample_fatalities_accuracy )
        

total_fatalities == 254.000
total_fatalities_success == 155.000
in_sample_fatalities_accuracy == 0.610


In [364]:
# Out sample accuracy

out_sample = pd.DataFrame(list(zip(clf.predict(X_test), Y_test)), columns=['predicted', 'actual'])

In [365]:
out_sample_survivors = out_sample[out_sample.actual == 1]
out_total_survivors = out_sample_survivors.count().actual
print("out-total_survivors == %.3f" %out_total_survivors)

out_sample_survivors_success = out_sample_survivors[out_sample_survivors.predicted == 1]
out_total_survivors_success = out_sample_survivors_success.count().predicted
print("out_total_survivors_success == %.3f" %out_total_survivors_success)

out_sample_survivors_accuracy = out_total_survivors_success/ out_total_survivors
print("out_sample_survivors_accuracy == %.3f" %out_sample_survivors_accuracy )

out-total_survivors == 175.000
out_total_survivors_success == 125.000
out_sample_survivors_accuracy == 0.714


In [366]:
out_sample_fatalities = out_sample[out_sample.actual == 0]
out_total_fatalities = out_sample_fatalities.count().actual
print("out-total_fatalities == %.3f" %out_total_fatalities)

out_sample_fatalities_success = out_sample_fatalities[out_sample_fatalities.predicted == 0]
out_total_fatalities_success = out_sample_fatalities_success.count().actual
print("out_total_fatalities_success == %.3f" %out_total_fatalities_success)

out_sample_fatalities_accuracy = out_total_fatalities_success/ out_total_fatalities
print("out_sample_fatalities_accuracy == %.3f" %out_sample_fatalities_accuracy )

out-total_fatalities == 244.000
out_total_fatalities_success == 195.000
out_sample_fatalities_accuracy == 0.799


## Comparing Accuracies with the accuracy of Decision Tree

In [391]:
rows = ['in_sample_survivor_accuracy','in_sample_fatalities_accuracy','out_sample_survivors_accuracy','out_sample_fatalities_accuracy']
NeuralNetValues = [in_sample_survivor_accuracy, in_sample_fatalities_accuracy, out_sample_survivors_accuracy,out_sample_fatalities_accuracy]
DecisionTreeValues = [0.535, 0.966, 0.571,0.957]
t = Table([rows,NeuralNetValues, DecisionTreeValues], names=('Rows','NeuralNetValues', 'DecisionTreeValues'), meta={'name': 'first table'})
t['NeuralNetValues'].format = '%.3f'



In [392]:
t


Rows,NeuralNetValues,DecisionTreeValues
str30,float64,float64
in_sample_survivor_accuracy,0.455,0.535
in_sample_fatalities_accuracy,0.610,0.966
out_sample_survivors_accuracy,0.714,0.571
out_sample_fatalities_accuracy,0.799,0.957
